In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
import json
from PIL import Image
import os

In [2]:
root_path = '/mnt/hdd/tanhz/clip_bk/res'
res_start_str='=> result'
keyword = 'accuracy'
ky = 'total'
metric = {
        "name": keyword,
        "regex": re.compile(fr"\* {keyword}: ([\.\deE+-]+)%"),
    }
all_results = dict()
for dataset_name in os.listdir(root_path):
    all_results[dataset_name] = dict()
    dataset_path = os.path.join(root_path, dataset_name)
    for trainer in os.listdir(dataset_path):
        all_results[dataset_name][trainer] = dict()
        sub_path = os.path.join(dataset_path, trainer)
        for sub in os.listdir(sub_path):
            all_results[dataset_name][trainer][sub] = dict()
            seed_path = os.path.join(sub_path, sub)
            outputs = []
            for seed in os.listdir(seed_path):
                output = dict()
                with open(os.path.join(seed_path, seed, 'log.txt'), 'r') as f:
                    lines = f.readlines()
                    res_signal = False
                    for line in lines:
                        line = line.strip()
                        if line == res_start_str:
                            res_signal = True
                        if not res_signal:
                            continue
                        # for metric in metrics:
                        match = metric["regex"].search(line)
                        if match:
                            regex = float(match.group(1))
                            output[metric['name']]=regex
                if output:
                    outputs.append(output)
            results = dict()
            for output in outputs:
                for key, value in output.items():
                    if key not in results:
                        results[key] = []
                    results[key].append(value)
            final_results = dict()
            for key, values in results.items():
                if key == 'accuracy':
                    avg = np.mean(values)
                    std = np.std(values)
                    final_results['avg'] = avg
                    final_results['std'] = std
                elif key == 'total':
                    avg = np.mean(values)
                    final_results['total'] = avg
            all_results[dataset_name][trainer][sub] = final_results

with open('/mnt/hdd/tanhz/clip_bk/res.json','w') as f:
    json.dump(all_results, f)


In [4]:
dataset_names = [
    'imagenet','caltech101','dtd','eurosat','fgvc_aircraft','food101','oxford_flowers','oxford_pets','stanford_cars','sun397','ucf101']
dataset_labels = [
    'ImageNet','Caltech101','DTD','EuroSAT','FGVCAircraft','Food101','Flowers102','OxfordPets','StanfordCars','SUN397','UCF101']

# colors = [
#         "#003147","#FF0097","#DC3023","#3DE1AD","#057748","#009ADE","#AF58BA","#FFC61E","#F28522","#A6761D","#443F90","#8F003B"
#     ]
# colors = [
#         "#003147","#104680","#317CB7","#6DADD1","#B6D7E8","#E9F1F4","#FBE3D5","#F6B293","#DC6D57","#B72230","#6D011F"
#     ]
# "#44045A"
colors = [
        "#44045A","#003147","#413E85","#104680","#317CB7",
        "#FFB703","#FB8302","#FF5100",
        "#D84527","#B72230","#6D011F"
    ]
markers = ['s','v','o','^','<','>','p','P','*','h','D','X','8']
trainers = ['ZeroshotCLIP','CoOp','CoCoOp','VPT','MaPLe',
            'ProGrad','KgCoOp','RPO',
            'PromptSRC','ProDA','TCP'
            ]
trainer_labels = ['CLIP(Zero-shot)','CoOp','CoCoOp','VPT','MaPLe',
            'ProGrad','KgCoOp','RPO',
            'PromptSRC','ProDA','TCP'
            ]
# ,'PLOTPP'
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 20
plt.rcParams['figure.dpi'] = 400
plt.rcParams['figure.dpi'] = 400
# colors = plt.get_cmap('hsv')

# plt.rcParams['font.weight'] = 'bold'
# plt.rcParams['font.style'] = 'italic'


集合结果

In [8]:
xs = dict()
xs['new'] = ['base','new1','new2','new3','new4','new5']
xs['newratio'] = ['base','new_ratio1','new_ratio2','new_ratio3','new_ratio4','new_ratio5']
x=dict()
x['t'] = ['0','0.2','0.4','0.6','0.8','1.0']
plt.rcParams['font.size'] = 20
for x_type in xs.keys():
    x_axis = xs[x_type]
    grid = plt.GridSpec(3,4,wspace=0.15,hspace=0.3,figure=plt.figure(figsize=(28,25)))
    p = q = 0
    for dataset_name,dataset_label in zip(dataset_names,dataset_labels):
        plt.subplot(grid[p,q])
        q += 1
        if q > 3:
            q = 0
            p += 1
        i = 0
        for trainer, trainer_label in zip(trainers,trainer_labels):
            y_axis_acc = []
            y_axis_std = []
            for sub in x_axis:
                y_axis_acc.append(all_results[dataset_name][trainer][sub]['avg'])
                y_axis_std.append(all_results[dataset_name][trainer][sub]['std'])
            y_axis_acc = np.array(y_axis_acc)
            y_axis_std = np.array(y_axis_std)
            plt.plot(x['t'], y_axis_acc, linestyle='--',marker=markers[i],markersize=7,color=colors[i] ,alpha=1, linewidth=1, label=trainer_label)
            plt.fill_between(x['t'], y_axis_acc-y_axis_std, y_axis_acc+y_axis_std, color=colors[i], alpha=0.1)
            # plt.errorbar(x_axis, y_axis_acc, yerr=y_axis_std, ecolor=colors[i], color=colors[i], elinewidth=1, capsize=5)
            i += 1
        plt.title(f'{dataset_label}')
        # _base2{x_type}
        plt.xlabel('t')
        plt.ylabel('Accuracy')
        if p == 2 and q == 3 :
            plt.legend(bbox_to_anchor=(1.15, 0), loc='lower left')
            # print(legend.get_children)
        if x_type=='newratio' and dataset_name=='sun397':
            plt.ylim(66,86)
            plt.yticks(np.arange(68,88,4))
        if x_type=='new' and dataset_name=='oxford_flowers':
            plt.ylim(63,99)
        if x_type=='newratio' and dataset_name=='oxford_flowers':
            plt.ylim(59,99)
        # plt.savefig(f'/mnt/hdd/tanhz/clip_bk/img/{dataset_name}_base2{x_type}.png',dpi=400)
        # plt.clf()
    plt.savefig(f'/mnt/hdd/tanhz/clip_bk/new_img_2/base2{x_type}-all.png',bbox_inches='tight', pad_inches=0.2)
    plt.savefig(f'/mnt/hdd/tanhz/clip_bk/new_img_2/base2{x_type}-all.eps',bbox_inches='tight', pad_inches=0.2)
    plt.savefig(f'/mnt/hdd/tanhz/clip_bk/new_img_2/base2{x_type}-all.pdf',format='pdf',bbox_inches='tight', pad_inches=0.2)
    plt.clf()
# plt.show()

NameError: name 'dataset_names' is not defined

<Figure size 2800x2500 with 0 Axes>

NewRatio - New

In [ ]:
xs = dict()
xs['new'] = ['base','new1','new2','new3','new4','new5']
xs['newratio'] = ['base','new_ratio1','new_ratio2','new_ratio3','new_ratio4','new_ratio5']
x_diff = ['new_ratio1-new1','new_ratio2-new2','new_ratio3-new3','new_ratio4-new4','new_ratio5-new5']
colors = plt.get_cmap('hsv')

# for dataset_name in os.listdir(root_path):
for dataset_name in os.listdir(root_path):

    fig = plt.figure(figsize=(8, 6))
    i = 0
    for trainer in all_results[dataset_name].keys():
        y_axis_accs = []
        for x_type in xs.keys():
            x_axis = xs[x_type]
            y_axis_acc = []
            for sub in x_axis:
                y_axis_acc.append(all_results[dataset_name][trainer][sub]['avg'])
            y_axis_acc = np.array(y_axis_acc)
            y_axis_accs.append(y_axis_acc)
        y_axis = np.array(y_axis_accs[1]-y_axis_accs[0])[1:]
        plt.plot(x_diff, y_axis, '*--',color=colors(i) ,alpha=0.5, linewidth=1, label=trainer)
        #     plt.fill_between(x_axis, y_axis_acc-y_axis_std, y_axis_acc+y_axis_std, color=colors(i), alpha=0.2)
            # plt.errorbar(x_axis, y_axis_acc, yerr=y_axis_std, ecolor=colors(i), color=colors(i), elinewidth=1, capsize=5)
        i += 30
        plt.xlabel(f'{dataset_name}_diff')
    plt.ylabel('differece')
    plt.legend(bbox_to_anchor=(1, 0), loc='lower left')
    # plt.ylim(85,100)
    # plt.savefig(f'/mnt/hdd/tanhz/clip_bk/img/{dataset_name}_base2{x_type}.png',dpi=400)
    # plt.clf()
    plt.show()

分数据集结果

In [14]:
xs = dict()
xs['new'] = ['base','new1','new2','new3','new4','new5']
xs['newratio'] = ['base','new_ratio1','new_ratio2','new_ratio3','new_ratio4','new_ratio5']
# colors = plt.get_cmap('hsv')
# plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 10
# plt.rcParams['font.weight'] = 'bold'
# plt.rcParams['font.style'] = 'italic'

for x_type in xs.keys():
    x_axis = xs[x_type]
    for dataset_name in os.listdir(root_path):
        i = 0
        y_axis_zero = []
        for sub in x_axis:
            y_axis_zero.append(all_results[dataset_name]['ZeroshotCLIP'][sub]['avg'])
        y_axis_zero = np.array(y_axis_zero)
        for trainer, trainer_label in zip(trainers,trainer_labels):
            y_axis_acc = []
            y_axis_std = []
            for sub in x_axis:
                y_axis_acc.append(all_results[dataset_name][trainer][sub]['avg'])
                y_axis_std.append(all_results[dataset_name][trainer][sub]['std'])
            y_axis_acc = np.array(y_axis_acc)
            y_axis_std = np.array(y_axis_std)
            # y_axis_minus = y_axis_acc - y_axis_zero
            plt.plot(x['t'], y_axis_acc,linestyle='--',marker=markers[i],markersize=4,color=colors[i] ,alpha=1, linewidth=1, label=trainer_label)
            plt.fill_between(x['t'], y_axis_acc-y_axis_std, y_axis_acc+y_axis_std, color=colors[i], alpha=0.1)
            # plt.errorbar(x_axis, y_axis_acc, yerr=y_axis_std, ecolor=colors[i], color=colors[i], elinewidth=1, capsize=5)
            i += 1
        plt.title(f'{dataset_label}_{x_type}')
        # _base2{x_type}
        plt.xlabel('t')
        plt.ylabel('Accuracy')
        plt.legend(bbox_to_anchor=(0, 0), loc='lower left',fontsize=7)
        # plt.ylim(85,100)
        # plt.savefig(f'/mnt/hdd/tanhz/clip_bk/img/compare_zero/{dataset_name}_base2{x_type}.png',dpi=400)
        plt.savefig(f'/mnt/hdd/tanhz/clip_bk/img/{dataset_name}_base2{x_type}.png',dpi=400)
        plt.clf()
    # plt.show()

<Figure size 2560x1920 with 0 Axes>

Metrics

In [6]:
def AUC(y_axis:np.ndarray):
    auc = (np.sum(y_axis[:-1]) + np.sum(y_axis[1:])) / (2 * (len(y_axis)-1))
    return auc

def WA(y_axis:np.ndarray):
    wa = np.min(y_axis)
    return wa

def base_acc(y_axis:np.ndarray):
    return y_axis[0]

def difference(y_axis:np.ndarray):
    return np.max(y_axis) - np.min(y_axis)

def EVM(y_axis:np.ndarray):
    evm = 0
    for j in range(len(y_axis)-1):
        evm += abs(y_axis[j+1] - y_axis[j])
    return evm

def VS(y_axis:np.ndarray):
    vs = 0
    for j in range(len(y_axis)-1):
        vs += (y_axis[j+1] - y_axis[j]) ** 2
    vs *= (len(y_axis) - 1)
    A = y_axis[-1] - y_axis[0]
    vs -= A ** 2
    return vs

def upper_down_area(y_axis:np.ndarray,y_axis_zero:np.ndarray):
    assert len(y_axis) == len(y_axis_zero)
    upper_area = 0
    down_area = 0
    length = len(y_axis)-1
    for i in range(length):
        det0 = y_axis[i] - y_axis_zero[i]
        det1 = y_axis[i+1] - y_axis_zero[i+1]
        if det0 * det1 >= 0:
            s = (det0 + det1) / 2 / length
            if s > 0:
                upper_area += s
            else:
                down_area += s
        else:
            s0 = (det0 ** 2) / (abs(det0) + abs(det1)) / 2 / length
            s1 = (det1 ** 2) / (abs(det0) + abs(det1)) / 2 / length
            if det0 > 0:
                upper_area += s0
                down_area += s1
            else:
                upper_area += s1
                down_area += s0
    return abs(upper_area),abs(down_area)


def metrics_cal(y_axis:np.ndarray,y_axis_zero:np.ndarray, metric:str):
    y_axis_small = y_axis / 100
    y_axis_zero_small = y_axis_zero / 100
    if metric == 'AUC':
        return AUC(y_axis_small)
    elif metric == 'WA':
        return WA(y_axis_small)
    elif metric == 'EVM':
        return EVM(y_axis_small)
    elif metric == 'VS':
        return VS(y_axis_small)
    elif metric == 'base_acc':
        return base_acc(y_axis_small)
    elif metric == 'difference':
        return difference(y_axis_small)
    elif metric == 'upper_area':
        upper_area, _ = upper_down_area(y_axis_small,y_axis_zero_small)
        return upper_area
    elif metric == 'down_area':
        _, down_area = upper_down_area(y_axis_small,y_axis_zero_small)
        return down_area
    elif metric == 'u-d':
        upper_area, down_area = upper_down_area(y_axis_small,y_axis_zero_small)
        if upper_area == 0:
            return 'inf'
        return down_area/upper_area

xs = dict()
xs['new'] = ['base','new1','new2','new3','new4','new5']
xs['newratio'] = ['base','new_ratio1','new_ratio2','new_ratio3','new_ratio4','new_ratio5']
metrics = ['base_acc','AUC', 'WA', 'EVM', 'VS', 'upper_area', 'down_area']
# trainers = ['ZeroshotCLIP','CoOp','CoCoOp','KgCoOp','MaPLe','ProDA','ProGrad','PromptSRC','TCP','VPT']

for x_type in xs.keys():
    x_axis = xs[x_type]
    ranks = []
    for dataset_name in dataset_names:
        # trainers = list(all_results[dataset_name].keys())
        df = pd.DataFrame(columns=metrics, index=trainer_labels)
        y_axis_acc_zero = []
        y_axis_std_zero = []
        for sub in x_axis:
            y_axis_acc_zero.append(all_results[dataset_name]['ZeroshotCLIP'][sub]['avg'])
            y_axis_std_zero.append(all_results[dataset_name]['ZeroshotCLIP'][sub]['std'])
        y_axis_acc_zero = np.array(y_axis_acc_zero)
        y_axis_std_zero = np.array(y_axis_std_zero)
        y_s = []
        for trainer, trainer_label in zip(trainers,trainer_labels):
            y_axis_acc = []
            y_axis_std = []
            for sub in x_axis:
                y_axis_acc.append(all_results[dataset_name][trainer][sub]['avg'])
                y_axis_std.append(all_results[dataset_name][trainer][sub]['std'])
            y_axis_acc = np.array(y_axis_acc)
            y_s.append(y_axis_acc)
            y_axis_std = np.array(y_axis_std)
            for metric in metrics:
                df.loc[trainer_label, metric] = metrics_cal(y_axis_acc,y_axis_acc_zero, metric)
        y_s = np.array(y_s)
        avg_rank =  np.mean(np.argsort(y_s.argsort(axis=0)[::-1], axis=0) + 1, axis=1)
        # df['rank'] = avg_rank
        # ranks.append(avg_rank)
        df.to_csv(f'/mnt/hdd/tanhz/clip_bk/statistic/metrics-{dataset_name}-{x_type}.csv')
    # ranks = np.array(ranks)
    # df_ranks = pd.DataFrame(ranks,columns=trainers, index=dataset_names)
    # df_ranks.loc['avg'] = np.mean(ranks, axis=0)
    # df_ranks.to_csv(f'/mnt/hdd/tanhz/clip_bk/statistic/ranks-{x_type}.csv')

In [23]:
def AUC(y_axis:np.ndarray):
    auc = (np.sum(y_axis[:-1]) + np.sum(y_axis[1:])) / (2 * (len(y_axis)-1))
    return auc

def WA(y_axis:np.ndarray):
    wa = np.min(y_axis)
    return wa

def base_acc(y_axis:np.ndarray):
    return y_axis[0]

def difference(y_axis:np.ndarray):
    return np.max(y_axis) - np.min(y_axis)

def EVM(y_axis:np.ndarray):
    evm = 0
    for j in range(len(y_axis)-1):
        evm += abs(y_axis[j+1] - y_axis[j])
    return evm

def VS(y_axis:np.ndarray):
    vs = 0
    for j in range(len(y_axis)-1):
        vs += (y_axis[j+1] - y_axis[j]) ** 2
    vs *= (len(y_axis) - 1)
    A = y_axis[-1] - y_axis[0]
    vs -= A ** 2
    return vs

def upper_down_area(y_axis:np.ndarray,y_axis_zero:np.ndarray):
    assert len(y_axis) == len(y_axis_zero)
    upper_area = 0
    down_area = 0
    length = len(y_axis)-1
    for i in range(length):
        det0 = y_axis[i] - y_axis_zero[i]
        det1 = y_axis[i+1] - y_axis_zero[i+1]
        if det0 * det1 >= 0:
            s = (det0 + det1) / 2 / length
            if s > 0:
                upper_area += s
            else:
                down_area += s
        else:
            s0 = (det0 ** 2) / (abs(det0) + abs(det1)) / 2 / length
            s1 = (det1 ** 2) / (abs(det0) + abs(det1)) / 2 / length
            if det0 > 0:
                upper_area += s0
                down_area += s1
            else:
                upper_area += s1
                down_area += s0
    return abs(upper_area),abs(down_area)


def metrics_cal(y_axis:np.ndarray,y_axis_zero:np.ndarray, metric:str):
    y_axis_small = y_axis / 100
    y_axis_zero_small = y_axis_zero / 100
    if metric == 'AUC':
        return AUC(y_axis_small)
    elif metric == 'WA':
        return WA(y_axis_small)
    elif metric == 'EVM':
        return EVM(y_axis_small)
    elif metric == 'VS':
        return VS(y_axis_small)
    elif metric == 'base_acc':
        return base_acc(y_axis_small)
    elif metric == 'difference':
        return difference(y_axis_small)
    elif metric == 'upper_area':
        upper_area, _ = upper_down_area(y_axis_small,y_axis_zero_small)
        return upper_area
    elif metric == 'down_area':
        _, down_area = upper_down_area(y_axis_small,y_axis_zero_small)
        return down_area
    elif metric == 'detaauc':
        return AUC(y_axis_small)-AUC(y_axis_zero_small)
    elif metric == 'd/u':
        upper_area, down_area = upper_down_area(y_axis_small,y_axis_zero_small)
        # if upper_area == 0:
        #     return 'inf'
        return upper_area-down_area

xs = dict()
xs['new'] = ['base','new1','new2','new3','new4','new5']
xs['newratio'] = ['base','new_ratio1','new_ratio2','new_ratio3','new_ratio4','new_ratio5']
# trainers = ['ZeroshotCLIP','CoOp','CoCoOp','KgCoOp','MaPLe','ProDA','ProGrad','PromptSRC','TCP','VPT']
metrics = ['detaauc','d/u']


for x_type in xs.keys():
    x_axis = xs[x_type]
    ranks = []
    dfs = None
    for dataset_name in dataset_names:
        # trainers = list(all_results[dataset_name].keys())
        df = pd.DataFrame(columns=metrics, index=trainer_labels)
        y_axis_acc_zero = []
        y_axis_std_zero = []
        for sub in x_axis:
            y_axis_acc_zero.append(all_results[dataset_name]['ZeroshotCLIP'][sub]['avg'])
            y_axis_std_zero.append(all_results[dataset_name]['ZeroshotCLIP'][sub]['std'])
        y_axis_acc_zero = np.array(y_axis_acc_zero)
        y_axis_std_zero = np.array(y_axis_std_zero)
        y_s = []
        for trainer, trainer_label in zip(trainers,trainer_labels):
            y_axis_acc = []
            y_axis_std = []
            for sub in x_axis:
                y_axis_acc.append(all_results[dataset_name][trainer][sub]['avg'])
                y_axis_std.append(all_results[dataset_name][trainer][sub]['std'])
            y_axis_acc = np.array(y_axis_acc)
            y_s.append(y_axis_acc)
            y_axis_std = np.array(y_axis_std)
            for metric in metrics:
                df.loc[trainer_label, metric] = metrics_cal(y_axis_acc,y_axis_acc_zero, metric)
        y_s = np.array(y_s)
        avg_rank =  np.mean(np.argsort(y_s.argsort(axis=0)[::-1], axis=0) + 1, axis=1)
        df['rank'] = avg_rank
        true_rank = np.argsort(np.argsort(avg_rank)) + 1
        df['true_rank'] = true_rank
        ranks.append(avg_rank)
        df.to_csv(f'/mnt/hdd/tanhz/clip_bk/statistic/robust-{dataset_name}-{x_type}.csv')
        if dfs is None:
            dfs = df
        else:
            dfs += df
    ranks = np.array(ranks)
    dfs /= len(dataset_names)
    dfs['true_rank'] = np.argsort(np.argsort(dfs['rank']))  + 1
    df_ranks = pd.DataFrame(ranks,columns=trainers, index=dataset_names)
    df_ranks.loc['avg'] = np.mean(ranks, axis=0)
    df_ranks.loc['true_rank'] = np.argsort(np.argsort(np.mean(ranks, axis=0))) + 1
    df_ranks.to_csv(f'/mnt/hdd/tanhz/clip_bk/statistic/ranks-{x_type}.csv')
    dfs.to_csv(f'/mnt/hdd/tanhz/clip_bk/statistic/robust-avg-{x_type}.csv')